<p style='text-align: justify;'>
<b>Ejercicio 1:</b> Implemente el algoritmo de retropropagación para un perceptrón multicapa de forma que se pueda elegir libremente la cantidad de capas de la red y de neuronas en cada capa. Pruébelo entrenando una red de estructura apropiada para resolver el problema XOR, con sus particiones de entrenamiento y prueba correspondientes (datos de la Guía de Trabajos Prácticos 1).</p>

#### <b>Librerías</b>

In [124]:
import random
import csv
import numpy as np

#### **Paso 1:** Inicialización
Determino la estructura de mi red neuronal: el número de capas y neuronas para cada capa. En la variable *cant_entradas*, el primer elemento son las entradas x1,x2,... del archivo, mientras que los próximos elementos me determinan la cantidad de neuronas por capa. 

In [125]:
cant_entradas = np.array([2,3,2,1]) # Variable a cambiar según mi red neuronal

cant_capas = len(cant_entradas) - 1
cant_salidas = cant_entradas[cant_capas]

Levantamos los datos del archivo .csv, separando en un vector para las entradas y otro para las salidas esperadas. En este caso, entrenamos y probamos el algoritmo con el problema XOR, para lo cual usaremos una red de dos capas y dos entradas x_i. La capa de entrada tendrá 2 neuronas y la capa de salida tendrá 1 neurona.


In [126]:
trn = np.loadtxt('./data/XOR_trn.csv',delimiter=',')

yd = []                         # Salida esperadas
for i in range(len(trn)):
    fila = trn[i]
    cant_e = cant_entradas[0]   # Cantidad de entradas
    yd.append(fila[cant_e])
    aux = [-1]                  # Añado entrada -1 correspondiente al umbral/sesgo
    for j in range(cant_e):
        aux.append(fila[j])
    trn[i] = aux                # Vector de entradas por patrón

Inicializo al azar las matrices de pesos de cada capa.

In [127]:
# Inicializo la matriz de pesos para cada una de las capas.
w = []
for i in range(len(cant_entradas)-1):
    w_aux = np.random.rand(cant_entradas[i+1],cant_entradas[i]+1)-0.5
    w.append(w_aux)     # Añado al vector de matrices de pesos

# Estructura de la matriz:
print(w)
# El elemento w_ij = w[i][j] de la matriz me da los pesos correspondientes a la neurona j de la capa i. 
# El elemento w_ij[k] me da el peso asociado a la entrada k de la neurona j de la capa i.

[array([[-0.0224112 , -0.20713099,  0.17189526],
       [ 0.06439677, -0.32651322, -0.43457923],
       [ 0.43931288, -0.08240215,  0.03705989]]), array([[-0.46001166,  0.14929832, -0.18748111,  0.01942142],
       [ 0.38082222,  0.19484938,  0.19490848,  0.00917571]]), array([[-0.45881635, -0.31778581, -0.49562906]])]


#### **Paso 2:** Entrenamiento
Continuando con el algoritmo, realizamos los pasos de propagación hacia adelante, propagación hacia atrás y adaptación de los pesos en distintas épocas para todas las capas de nuestra red neuronal. Realizamos una pasada de aprendizaje y otra de validación para evaluar el desempeño del mismo.

In [128]:
# DATOS PARA EL ALGORITMO:
epoca = 0               # Contador para época actual
epoca_max = 50          # Máximo de iteraciones
mu = 0.01               # Velocidad de aprendizaje
b = 1                   # Constante b para sismóidea
perc_error_max = 0.05   # Porcentaje máximo de error
errores_por_epoca = []
mse_por_epoca = []

# TODO: Agregar gráficas dinámicas

while (epoca < epoca_max):  
    
    #--------------------------------#
    #--------- Aprendizaje ----------#
    #--------------------------------#

    for patron in range(len(trn)):

        # PROPAGACIÓN HACIA ADELANTE: Obtengo la salida de las capas y las propago como entradas de las próximas
        entradas = trn[patron]          # La primera capa tiene las entradas en el archivo .csv
        for i in range(cant_capas):
            v = w[i]@entradas                  # Producto interno de pesos y entradas
            y[i] = 2/(1+np.exp(-b*v))-1        # Salida con función de activación
            entradas = np.hstack((-1,y[i]))    # Entrada de la próxima capa es la salida de esta capa
        
        # PROPAGACIÓN HACIA ATRÁS: Obtengo el delta de la capa de salida y lo propago a las capas anteriores
        error = yd[patron] - y[-1]                    
        delta[-1]=error*(1/2)*(1+y[-1])*(1-y[-1])       # Con el error, obtengo el delta de la capa de salida
        for i in range(cant_capas-1,0,-1):
            w_i = w[i][:,1:].T                          # No tomo el peso w0 (umbral) porque no tiene delta para propagar
            d = np.dot(w_i,delta[i])
            delta[i-1] = d*(1/2)*(1+y[i-1])*(1-y[i-1])  # Con los pesos de la capa i obtenemos el delta de la capa i-1
        
        # ACTUALIZAR LOS PESOS: Ajusto los pesos con la velocidad de aprendizaje, la entrada y su delta.
        entradas = trn[patron]          # La primera capa tiene las entradas en el archivo .csv
        for i in range(cant_capas):
            delta_peso = mu*(np.outer(delta[i],entradas))
            w[i] += delta_peso
            entradas = np.hstack((-1,y[i]))    # Entrada para próxima capa es la salida de esta
        
    #--------------------------------#
    #---------- Evaluación ----------#
    #--------------------------------#
    cont_errores = 0    # Contador de errores
    cont_mse = 0        # Contador para error cuadrático medio

    for patron in range(len(trn)): 

        # PROPAGACIÓN HACIA ADELANTE: Obtengo la salida de las capas y las propago como entradas de las próximas
        entradas = trn[patron]          # La primera capa tiene las entradas en el archivo .csv
        for i in range(cant_capas):
            v = w[i]@entradas                  # Producto interno de pesos y entradas
            y[i] = 2/(1+np.exp(-b*v)) - 1      # Salida con función de activación
            entradas = np.hstack((-1,y[i]))    # Entrada de la próxima capa es la salida de esta capa

        # CODIFICACIÓN: Función signo
        if (y[-1] < 0): yc = -1
        else: yc = 1

        # Actualizo contadores de error:
        if(yd[patron] != yc): cont_errores += 1
        cont_mse += np.sum(np.square(yd[patron]-yc)) 

    # Actualizo arrays para grafica de error:
    mse = cont_mse/len(trn)
    mse_por_epoca.append(mse)
    errores_por_epoca.append(cont_errores)
    # Porcentaje de error para criterio de parada:
    perc_error = cont_errores*100/len(trn)
    if(perc_error < perc_error_max): break

    epoca += 1

print('Finalizó el entrenamiento en la época ',epoca,' con ',cont_errores,'/',len(trn),' errores')

Finalizó el entrenamiento en la época  29  con  0 / 2000  errores


#### **Paso 3:** Prueba
Una vez obtenidos los pesos mediante el entrenamiento, pasamos otro dataset por la red neuronal para probarla. Comenzamos levantando el archivo .csv de la misma forma que para el entrenamiento.

In [129]:
tst = np.loadtxt('./data/XOR_tst.csv',delimiter=',')

yd = []                         # Salidas esperadas
for i in range(len(tst)):
    fila = tst[i]
    cant_e = cant_entradas[0]   # Cantidad de entradas
    yd.append(fila[cant_e])     
    aux = [-1]                  # Añado entrada -1 correspondiente al umbral/sesgo
    for j in range(cant_e):
        aux.append(fila[j])
    tst[i] = aux                # Vector de entradas por patrón

Realizamos la propagación hacia adelante, manteniendo un contador de errores para determinar la eficiencia del entrenamiento.

In [130]:
y = np.empty(cant_capas,dtype=object)   # Vector de salidas
cont_errores = 0                        # Contador de errores

for patron in range(len(tst)): 
    entradas = tst[patron]
    for i in range(cant_capas):
        v = w[i]@entradas                   # Producto interno de pesos y entradas
        y[i] = 2/(1+np.exp(-b*v)) - 1       # Salida con función de activación
        entradas = np.hstack((-1,y[i]))     # La entrada de la próxima capa es la salida de la actual.

    if (y[-1] < 0): yc = -1
    else: yc = 1
    if(yd[patron] != yc): cont_errores += 1

print('Finalizó la prueba con ',cont_errores,'/',len(tst),' errores.')

Finalizó la prueba con  0 / 200  errores.
